In [19]:
#Import library
import pandas as pd
import gurobipy as gp 
from gurobipy import GRB
import logging

In [2]:
# Parameters

years = [1, 2, 3]
skills = ['s1', 's2', 's3']

curr_workforce = {'s1': 2000, 's2': 1500, 's3': 1000}
demand = {
    (1, 's1'): 1000,
    (1, 's2'): 1400,
    (1, 's3'): 1000,
    (2, 's1'): 500,
    (2, 's2'): 2000,
    (2, 's3'): 1500,
    (3, 's1'): 0,
    (3, 's2'): 2500,
    (3, 's3'): 2000
}
rookie_attrition = {'s1': 0.25, 's2': 0.20, 's3': 0.10}
veteran_attrition = {'s1': 0.10, 's2': 0.05, 's3': 0.05}
demoted_attrition = 0.50
max_hiring = {
    (1, 's1'): 500,
    (1, 's2'): 800,
    (1, 's3'): 500,
    (2, 's1'): 500,
    (2, 's2'): 800,
    (2, 's3'): 500,
    (3, 's1'): 500,
    (3, 's2'): 800,
    (3, 's3'): 500
}
max_overmanning = 150
max_parttime = 50
parttime_cap = 0.50
max_train_unskilled = 200
max_train_semiskilled = 0.25

training_cost = {'s1': 400, 's2': 500}
layoff_cost = {'s1': 200, 's2': 500, 's3': 500}
parttime_cost = {'s1': 500, 's2': 400, 's3': 400}
overmanning_cost = {'s1': 1500, 's2': 2000, 's3': 3000}

In [ ]:
manpower = gp.Model('Manpower planning')

## Decision variables: Define the decisions variable; all variables are non-negative integers
hire = manpower.addVars(years, skills, ub=max_hiring,vtype=GRB.INTEGER, name="Hire")
part_time = manpower.addVars(years, skills, ub=max_parttime,
                          name="Part_time")
workforce = manpower.addVars(years, skills,vtype=GRB.INTEGER, name="Available")
layoff = manpower.addVars(years, skills, vtype=GRB.INTEGER,name="Layoff")
excess = manpower.addVars(years, skills,vtype=GRB.INTEGER, name="Overmanned")
train = manpower.addVars(years, skills, skills, vtype=GRB.INTEGER, name="Train")

## Define constraints:
#1. 

In [21]:
#1.1 & 1.2 Balance

Balance = manpower.addConstrs(
    (workforce[year, level] == (1-veteran_attrition[level])*(curr_workforce[level] if year == 1 else workforce[year-1, level])
    + (1-rookie_attrition[level])*hire[year, level] + gp.quicksum((1- veteran_attrition[level])* train[year, level2, level]
                                                        -train[year, level, level2] for level2 in skills if level2 < level)
    + gp.quicksum((1- demoted_attrition)* train[year, level2, level] -train[year, level, level2] for level2 in skills if level2 > level)
    - layoff[year, level] for year in years for level in skills), "Balance")

In [22]:
#2.1 & 2.2  Unskilled training
UnskilledTrain1 = manpower.addConstrs((train[year, 's1', 's2'] <= max_train_unskilled for year in years), "Unskilled_training1")
UnskilledTrain2 = manpower.addConstrs((train[year, 's1', 's3'] == 0 for year in years), "Unskilled_training2")

In [23]:
#3. Semi-skilled training

SemiskilledTrain = manpower.addConstrs((train[year,'s2', 's3'] <= max_train_semiskilled * workforce[year,'s3'] for year in years), "Semiskilled_training")

In [24]:
#4. Overmanning
Overmanning = manpower.addConstrs((excess.sum(year, '*') <= max_overmanning for year in years), "Overmanning")

In [25]:
#5. Demand
Demand = manpower.addConstrs((workforce[year, level] ==
     demand[year,level] + excess[year, level] + parttime_cap * part_time[year, level]
                     for year in years for level in skills), "Requirements")

In [30]:
#0.1 Objective Function: Minimize layoffs
obj2 =  gp.quicksum((training_cost[level]*train[year, level, skills[skills.index(level)+1]] if level < 's3' else 0)
                + layoff_cost[level]*layoff[year, level]
                + parttime_cost[level]*part_time[year, level]
                + overmanning_cost[level] * excess[year, level] for year in years for level in skills)
manpower.setObjective(obj2, GRB.MINIMIZE)

In [31]:
manpower.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 30 rows, 72 columns and 117 nonzeros
Model fingerprint: 0x0bb17cd3
Variable types: 9 continuous, 63 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [2e+02, 3e+03]
  Bounds range     [5e+01, 8e+02]
  RHS range        [2e+02, 3e+03]

MIP start from previous solve produced solution with objective 508700 (0.00s)
Loaded MIP start from previous solve with objective 508700

Presolve removed 16 rows and 25 columns
Presolve time: 0.00s
Presolved: 14 rows, 47 columns, 89 nonzeros
Variable types: 0 continuous, 47 integer (0 binary)

Root relaxation: objective 4.986773e+05, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     W

In [28]:
import numpy as np
rows = years.copy()
columns = skills.copy()
hire_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level in hire.keys():
    if (abs(hire[year, level].x) > 1e-6):
        hire_plan.loc[year, level] = np.round(hire[year, level].x, 1)
hire_plan

,s1,s2,s3
1,0.0,0.0,60.0
2,0.0,798.0,495.0
3,0.0,799.0,500.0


In [29]:
rows = years.copy()
columns = ['{0} to {1}'.format(level1, level2) for level1 in skills for level2 in skills if level1 != level2]
train_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level1, level2 in train.keys():
    col = '{0} to {1}'.format(level1, level2)
    if (abs(train[year, level1, level2].x) > 1e-6):
        train_plan.loc[year, col] = np.round(train[year, level1, level2].x, 1)
train_plan

,s1 to s2,s1 to s3,s2 to s1,s2 to s3,s3 to s1,s3 to s2
1,0.0,0.0,25.0,0.0,3.0,0.0
2,148.0,0.0,0.0,109.0,0.0,0.0
3,104.0,0.0,2.0,140.0,0.0,8.0


In [32]:
rows = years.copy()
columns = skills.copy()
layoff_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level in layoff.keys():
    if (abs(layoff[year, level].x) > 1e-6):
        layoff_plan.loc[year, level] = np.round(layoff[year, level].x, 1)
layoff_plan

,s1,s2,s3
1,814.0,0.0,0.0
2,252.0,0.0,0.0
3,347.0,0.0,0.0


In [33]:
rows = years.copy()
columns = skills.copy()
parttime_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level in part_time.keys():
    if (abs(part_time[year, level].x) > 1e-6):
        parttime_plan.loc[year, level] = np.round(part_time[year, level].x, 1)
parttime_plan

,s1,s2,s3
1,0.0,0.0,2.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0


In [34]:
rows = years.copy()
columns = skills.copy()
excess_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level in excess.keys():
    if (abs(excess[year, level].x) > 1e-6):
        excess_plan.loc[year, level] = np.round(excess[year, level].x, 1)
excess_plan

,s1,s2,s3
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0


In [35]:
manpower.write("manpower-planning-output.sol")

In [ ]:
gp.setParam("Cuts", 2)